<a href="https://colab.research.google.com/github/sheykh-8/TTS/blob/main/speech_brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install speechbrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

# Tacotron2 Example:

In [11]:
import torchaudio
from speechbrain.inference.TTS import Tacotron2
from speechbrain.inference.vocoders import HIFIGAN
from speechbrain.inference.vocoders import DiffWaveVocoder

In [9]:
diffwave = DiffWaveVocoder.from_hparams(source="speechbrain/tts-diffwave-ljspeech", savedir="tmpdir_vocoder")

hyperparams.yaml:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

diffwave.ckpt:   0%|          | 0.00/10.6M [00:00<?, ?B/s]

In [14]:
# Intialize TTS (tacotron2) and Vocoder (HiFIGAN)
tacotron2 = Tacotron2.from_hparams(source="speechbrain/tts-tacotron2-ljspeech", savedir="tmpdir_tts")
hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech", savedir="tmpdir_vocoder_hifi")

hyperparams.yaml:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


generator.ckpt:   0%|          | 0.00/55.8M [00:00<?, ?B/s]

In [15]:
# Running the TTS
mel_output, mel_length, alignment = tacotron2.encode_text("Good Morning! said Bilbo, and he meant it. The sun was shining, and the grass was very green. But Gandalf looked at him from under long bushy eyebrows that stuck out further than the brim of his shady hat.")

In [25]:
another_mel, ml, align = tacotron2.encode_text("The sun was shining, and the grass was very green.")

In [26]:
ml

tensor([284], dtype=torch.int32)

In [27]:
# Running Vocoder (spectrogram-to-waveform)
waveforms = hifi_gan.decode_batch(another_mel)

In [23]:
type(mel_output)

mel_output.shape

torch.Size([1, 80, 1000])

In [19]:
waveform_diffwave = diffwave.decode_batch(mel_output, hop_len=256)

KeyboardInterrupt: 

In [28]:
# Save the waverform
torchaudio.save('example_TTS_short.wav',waveforms.squeeze(1), 22050)

# Fastspeech2 Example:

In [29]:
from speechbrain.inference.TTS import FastSpeech2

In [30]:
# tmpdir_tts = getfixture('tmpdir')
fastspeech2 = FastSpeech2.from_hparams(source="speechbrain/tts-fastspeech2-ljspeech", savedir="tmpdir")

hyperparams.yaml:   0%|          | 0.00/3.41k [00:00<?, ?B/s]

spn_predictor.ckpt:   0%|          | 0.00/76.4M [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/181M [00:00<?, ?B/s]

hyperparams.yaml:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/129M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [33]:
mel_outputs, durations, pitch, energy = fastspeech2.encode_text(["The sun was shining, and the grass was very green."])


In [ ]:
mel_outputs, durations, pitch, energy = fastspeech2.encode_text(["Good Morning! said Bilbo, and he meant it.", "The sun was shining, and the grass was very green.", "But Gandalf looked at him from under long bushy eyebrows that stuck out further than the brim of his shady hat."])


In [35]:
mel_outputs.shape

torch.Size([1, 80, 267])

In [34]:

waveforms = hifi_gan.decode_batch(mel_outputs)

In [36]:
torchaudio.save('example_TTS_fastspeech1.wav', waveforms.squeeze(1), 22050)

# Fastspeech2 with internal alignments

In [ ]:
from speechbrain.inference.TTS import FastSpeech2InternalAlignment

In [ ]:
fastspeech2 = FastSpeech2InternalAlignment.from_hparams(source="speechbrain/tts-fastspeech2-internal-alignment-ljspeech", savedir="tmpdir_tts_alignments")

hyperparams.yaml:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/255M [00:00<?, ?B/s]

In [ ]:
mel_outputs, durations, pitch, energy = fastspeech2.encode_text(["Good Morning! said Bilbo, and he meant it. The sun was shining, and the grass was very green. But Gandalf looked at him from under long bushy eyebrows that stuck out further than the brim of his shady hat."])

waveforms = hifi_gan.decode_batch(mel_outputs)

In [ ]:
torchaudio.save('example_TTS_fastspeech2.wav', waveforms.squeeze(1), 22050)

# Experiments

In [39]:
import torch

In [42]:
# lets load the weights and peek around a little bit to see what's happenning:

model = torch.load('tmpdir_tts/model.ckpt', map_location=torch.device('cpu'))

In [43]:
type(model)

collections.OrderedDict

In [44]:
model

OrderedDict([('embedding.weight',
              tensor([[-3.0306e-03, -1.3483e-03, -7.7179e-04,  ...,  4.7210e-04,
                       -1.4310e-02,  1.4095e-04],
                      [-3.2937e-03,  5.8646e-03, -6.9003e-03,  ...,  1.3194e-03,
                       -6.2323e-03, -7.1334e-04],
                      [-2.6949e-03,  6.4383e-03,  1.6325e-03,  ..., -7.9077e-03,
                       -3.7150e-03, -5.3214e-05],
                      ...,
                      [ 6.8459e-40,  6.2043e-40, -5.2595e-40,  ..., -6.6856e-40,
                       -8.3492e-40,  7.1150e-40],
                      [ 3.9291e-41, -5.1281e-40,  3.4120e-40,  ..., -8.0577e-40,
                       -1.8515e-40,  5.5929e-40],
                      [-1.0894e-39,  1.0751e-39, -1.5310e-40,  ..., -3.1662e-40,
                        7.3239e-40, -3.4389e-40]])),
             ('encoder.convolutions.0.0.conv.weight',
              tensor([[[-2.3780e-03,  4.2240e-03, -1.3523e-03, -4.0996e-04,  5.2921e-03],
      

Lets try so save a sample inference as a json file and then load it to be used with a vocoder.

In [50]:
import json

In [58]:
mel_output.shape

torch.Size([1, 80, 1000])

In [60]:
mel_output.to('cpu')

inference_data = mel_output.tolist()

with open("sample_inference.json", "w") as f:
  json.dump(inference_data, f)

Now lets load the data back and try to convert it to a tensor and use it to generate audio using the vocoder

In [62]:
with open("sample_inference.json", "r") as f:
  raw_data = json.load(f)

  tensor = torch.tensor(raw_data)

  print(tensor.shape)

  waveforms = hifi_gan.decode_batch(tensor)

  torchaudio.save('example_TTS_from_json.wav', waveforms.squeeze(1), 22050)

torch.Size([1, 80, 1000])
